In [23]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [19]:
# Setup training data
train_data = datasets.FashionMNIST(
    root="data", # where to download data to?
    train=True, # get training data
    download=True, # download data if it doesn't exist on disk
    transform=ToTensor(), # images come as PIL format, we want to turn into Torch tensors
    target_transform=None # you can transform labels as well
)

# Setup testing data
test_data = datasets.FashionMNIST(
    root="data",
    train=False, # get test data
    download=True,
    transform=ToTensor()
)

0it [00:00, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz


0it [00:00, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz


0it [00:00, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


/Users/michael.todisco/.pyenv/versions/anaconda3-2021.05/lib/python3.8/site-packages/torchvision/datasets/mnist.py:335: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659484782168/work/torch/csrc/utils/tensor_numpy.cpp:178.)
  return torch.from_numpy(parsed).view(length, num_rows, num_cols)


In [24]:
# Setup the batch size hyperparameter
BATCH_SIZE = 32

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(train_data, # dataset to turn into iterable
    batch_size=BATCH_SIZE, # how many samples per batch? 
    shuffle=True # shuffle data every epoch?
)

test_dataloader = DataLoader(test_data,
    batch_size=BATCH_SIZE,
    shuffle=False # don't necessarily have to shuffle the testing data
)

# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7fadcc730b50>, <torch.utils.data.dataloader.DataLoader object at 0x7fadcc7306a0>)
Length of train dataloader: 1875 batches of 32
Length of test dataloader: 313 batches of 32


In [141]:
class TorchVision(nn.Module):
    def __init__(self,
                 input_shape,
                 hidden_units,
                 output_shape,
                ):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels = input_shape,
                      out_channels=hidden_units,
                      kernel_size = 3,
                      stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size = 3,
                      stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units*5*5, output_shape)
        )
        
    def forward(self, x):
        x = self.block_1(x)
        # print(f'block 1 shape: {x.shape}')
        x = self.block_2(x)
        # print(f'block 2 shape: {x.shape}')
        x = self.classifier(x)
        # print(f'output shape: {x.shape}')
        return x

In [142]:
model = TorchVision(input_shape=1, hidden_units=10, output_shape=len(train_data.classes))

In [143]:
torch.manual_seed(42)

# Create sample batch of random numbers with same size as image batch
# images = torch.randn(size=(32, 1, 64, 64)) # [batch_size, color_channels, height, width]
# test_image = images[0]
# model(test_image.unsqueeze(dim=0))

In [144]:
from sklearn.metrics import accuracy_score

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               optimizer: torch.optim.Optimizer,
               loss_fn: torch.nn.Module,
               device='cpu'
              ):
    model.train()
    
    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(dataloader):
        
        #Forward pass
        logits = model(X)
        y_preds = torch.softmax(logits, dim=1).argmax(dim=1)
        
        # Calculate loss
        loss = loss_fn(logits, y)
        train_loss += loss.item()
        accuracy = accuracy_score(y_preds, y)
        train_acc += accuracy
        
        # Zero out gradient
        optimizer.zero_grad()
        
        # back prop
        loss.backward()
        
        # step
        optimizer.step()
        
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    
    return train_loss, train_acc

In [155]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              optimizer: torch.optim.Optimizer,
              device='cpu'):
    
    model.eval()
    
    test_loss, test_acc = 0, 0
    
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            
            logits = model(X)
            y_preds = torch.softmax(logits, dim=1).argmax(dim=1)
            
            test_loss += loss_fn(logits, y).item()
            test_acc += accuracy_score(y_preds, y)
            
        test_loss = test_loss / len(dataloader)
        test_acc = test_acc / len(dataloader)
        
        return test_loss, test_acc

In [167]:
from tqdm.auto import tqdm

def train(model,
          train_dataloader,
          test_dataloader,
          loss_fn,
          optimizer,
          epochs,
          device
         ):
    
    results = {
        'train_loss': [],
        'train_acc': [],
        'test_loss': [],
        'test_acc': [],
    }
    
    for epoch in tqdm(range(1, epochs+1)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           optimizer=optimizer,
                                           loss_fn=loss_fn
                                          )
        
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        optimizer=optimizer,
                                        loss_fn=loss_fn
                                       )
        
        print(f"epoch: {epoch} | train loss: {train_loss:.4f} | train acc: {train_acc:.4f} | test loss: {test_loss:.4f} |  test acc {test_acc:.4f}")
        

In [168]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
train(model,
      train_dataloader,
      test_dataloader,
      loss_fn,
      optimizer,
      epochs=10,
      device='cpu'
     )

  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 1 | train loss: 0.2600 | train acc: 0.9051 | test loss: 0.3158 |  test acc 0.8883
epoch: 2 | train loss: 0.2588 | train acc: 0.9051 | test loss: 0.3365 |  test acc 0.8782
epoch: 3 | train loss: 0.2574 | train acc: 0.9065 | test loss: 0.3190 |  test acc 0.8890
epoch: 4 | train loss: 0.2568 | train acc: 0.9053 | test loss: 0.3167 |  test acc 0.8888
epoch: 5 | train loss: 0.2567 | train acc: 0.9063 | test loss: 0.3188 |  test acc 0.8891
epoch: 6 | train loss: 0.2553 | train acc: 0.9071 | test loss: 0.3353 |  test acc 0.8829
epoch: 7 | train loss: 0.2555 | train acc: 0.9067 | test loss: 0.3169 |  test acc 0.8903
epoch: 8 | train loss: 0.2548 | train acc: 0.9059 | test loss: 0.3251 |  test acc 0.8878
epoch: 9 | train loss: 0.2544 | train acc: 0.9064 | test loss: 0.3256 |  test acc 0.8834
epoch: 10 | train loss: 0.2537 | train acc: 0.9065 | test loss: 0.3160 |  test acc 0.8910
